# Algorithmic Trading System Demonstration

System for time series prediction

## Database Exploratory Analysis

In [ ]:
%%capture
%run ./DatabaseExplore.ipynb

### Load bonds, commodities and index data from database

**Choose the date range for which you need to analyze the data for. (Format: YYYY-MM-DD)**  
**Add the stock symbols in the 'symbols' list to choose stocks (case sensitive)**  
*[List of stocks](./stocksList.ipynb)*

In [ ]:
# Input parameters
start_date = '2021-06-28'
end_date = '2024-06-27'
symbols = ['AAPL','NVDA','MSFT','CSCO','AMZN','GOOGL']

# Fetch from database
indexes, commodities, bonds = getCommodities(start_date, end_date)
stocks, topStocks = getStocks(symbols, start_date, end_date)

### Plot indices graph

In [ ]:
plotIndexes(indexes)

### Plot commodities graph

In [ ]:
plotCommodities(commodities)

### Plot bonds graph

In [ ]:
plotBond(bonds)

### Plot Top 10 stocks

In [ ]:
plotTopStocks(topStocks)

### Plot selected stocks graphs

In [ ]:
plotSelectedStocks(stocks)

## Stock price forecast

In [4]:
%run ./Prediction.ipynb

ModuleNotFoundError: No module named 'sklearn'

ModuleNotFoundError: No module named 'sklearn'

### Load dataset and filter

In [ ]:
# Read data from csv
df = loadDataset()
# Filter dataframe with selected symbols
filterDataset(symbols, df)
# Preprocess dataframe fields
df, DATETIME = preprocess(df)

### Train model and generate prediction

In [ ]:
# Input data
trainingDateStart = '2019-06-26'
predictionDateStart = '2024-05-15'
predictionDateEnd = '2024-06-15'
daysToPredict = 5

# Generate prediction
result = trainModel(trainingDateStart,predictionDateStart, predictionDateEnd, daysToPredict)

## Result Analysis

In [ ]:
plotPrediction(result)

## Metrics Analysis

In [ ]:
metrics_df = calculateMetrics(result)
plotMetrics(metrics_df)